# Loader ARM64

Last edited: 2025-06-14

Environment: Termux, SSH server, proot-distro Ubuntu 22.04 AArch64, GNU tools, and JupyterLab server.

For this work, the idea is to write a generic loader for pure binary files (without the ELF overhead), which will be used in this and other projects as well.

In [1]:
%%writefile load.c
#include <stdio.h>
#include <stdlib.h>
#include <sys/mman.h>   // For mmap and munmap
#include <sys/stat.h>   // For stat and fstat
#include <fcntl.h>      // For open
#include <unistd.h>     // For read, close

// Define the fixed allocation size
#define ALLOC_SIZE 4096

int main(int argc, char *argv[]) {
    void *exec_mem;
    int fd;
    struct stat st;
    size_t bytes_to_read;

    // Check for correct usage
    if (argc != 2) {
        fprintf(stderr, "Usage: %s <path_to_raw_binary>\n", argv[0]);
        return 1;
    }

    const char *bin_path = argv[1];

    // Open the raw binary file
    fd = open(bin_path, O_RDONLY);
    if (fd == -1) {
        perror("Error opening binary file");
        return 1;
    }

    // Get file size to determine how many bytes to read (up to ALLOC_SIZE)
    if (fstat(fd, &st) == -1) {
        perror("Error getting file information");
        close(fd);
        return 1;
    }

    // Determine how many bytes to read: min(file_size, ALLOC_SIZE)
    bytes_to_read = (st.st_size < ALLOC_SIZE) ? st.st_size : ALLOC_SIZE;

    if (bytes_to_read == 0) {
        fprintf(stderr, "Error: Binary file is empty or too small.\n");
        close(fd);
        return 1;
    }

    // Allocate memory with read, write, and execute permissions
    // PROT_READ  : Allows reading
    // PROT_WRITE : Allows writing (crucial for self-modifying code)
    // PROT_EXEC  : Allows execution
    // MAP_PRIVATE|MAP_ANONYMOUS: Private, anonymous memory mapping
    exec_mem = mmap(NULL, ALLOC_SIZE, PROT_READ | PROT_WRITE | PROT_EXEC,
                    MAP_PRIVATE | MAP_ANONYMOUS, -1, 0);

    if (exec_mem == MAP_FAILED) {
        perror("Error allocating executable memory with mmap");
        close(fd);
        return 1;
    }

    // Read the file content into the allocated memory
    ssize_t bytes_read_actual = read(fd, exec_mem, bytes_to_read);
    if (bytes_read_actual == -1) {
        perror("Error reading binary file into memory");
        munmap(exec_mem, ALLOC_SIZE);
        close(fd);
        return 1;
    }
    if ((size_t)bytes_read_actual != bytes_to_read) {
        fprintf(stderr, "Warning: Could not read all expected bytes (%zu of %zu).\n",
                bytes_read_actual, bytes_to_read);
    }

    close(fd); // Close the file descriptor, content is in memory

    // Cast the executable memory to a function pointer and jump to it.
    // Execution of the binary starts from its very first byte loaded into memory.
    void (*binary_func)() = (void(*)())exec_mem;
    binary_func();

    // This line will only be reached if the loaded binary doesn't exit the process.
    // Clean up allocated memory if control returns here.
    munmap(exec_mem, ALLOC_SIZE);

    return 0; // If the binary doesn't call _exit
}

Overwriting load.c


In [2]:
%%bash
gcc -Wl,-z,norelro -o load  load.c
strip --strip-section-headers  load

In [3]:
! gcc -g -o load load.c

In [4]:
! mv load /usr/local/bin/

We'll place our custom loader in /usr/local/bin/ with other local executables. This loader will then execute generic machine code binaries.

Below is our first code to test the loader:

In [5]:
%%writefile tiny.s
    .arch armv8-a
    .section .text
    .globl _start
    .type _start, %function

_start:
    mov     x8, #93      // Syscall number for _exit (93)
    mov     x0, #42      // Exit code argument
    svc     #0           // Perform syscall

Writing tiny.s


In [6]:
! as tiny.s -o tiny.o

In [7]:
! objcopy -O binary -j .text tiny.o tiny.bin

In [8]:
! load tiny.bin ; echo $?

42


In [9]:
! wc -c tiny.bin

12 tiny.bin


The file is only 12 bytes long and contains only the executable code, without the ELF overhead.

In [37]:
! hexdump -C tiny.bin

00000000  a8 0b 80 d2 40 05 80 d2  01 00 00 d4              |....@.......|
0000000c


In [38]:
! objdump -d tiny.o


tiny.o:     file format elf64-littleaarch64


Disassembly of section .text:

0000000000000000 <_start>:
   0:	d2800ba8 	mov	x8, #0x5d                  	// #93
   4:	d2800540 	mov	x0, #0x2a                  	// #42
   8:	d4000001 	svc	#0x0
